### 01 - Rutas completas de órdenes (header + operaciones)

Objetivo: explorar `ordenes_header.csv` y `produccion_operaciones.csv`, reconstruir la secuencia de fases por orden y detectar órdenes con ruta completa (ej. empieza en curvado y acaba en rectificado/control final).

In [231]:
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter
import re

pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 200)

RAW = Path('../data/raw')
PATH_HEADER = RAW / 'ordenes_header.csv'
PATH_OPS = RAW / 'produccion_operaciones.csv'

header = pd.read_csv(PATH_HEADER)
ops = pd.read_csv(PATH_OPS)

print('Header shape:', header.shape)
print(header.head(3))
print('Ops shape:', ops.shape)
print(ops.head(3))


Header shape: (327, 8)
  work_order_id  ref_id  familia     cliente  qty_plan fecha_lanzamiento due_date planta_inicio
0       24/0749  081901  PENDING  Schaeffler      2500        2025-01-23  PENDING       PENDING
1       24/0717  081906  PENDING  Schaeffler      5000        2025-01-23  PENDING       PENDING
2       24/0672  081290  PENDING  Schaeffler      4427        2025-01-24  PENDING       PENDING
Ops shape: (64555, 23)
  work_order_id work_order_id_raw         op_id  machine_id machine_name    planta       op_text    ref_id ref_id_raw               ts_ini               ts_fin  duracion_min  piezas_ok  \
0       24/0658           24/0658  SOLDADURA-RE         515    SOLDADORA  Zaldibar  SOLDADURA-RE   81000.0    81000.0  2025-01-06T21:47:00  2025-01-06T21:50:00           3.0          0   
1       24/0658           24/0658  SOLDADURA-RE         515    SOLDADORA  Zaldibar  SOLDADURA-RE   81000.0    81000.0  2025-01-06T21:48:00  2025-01-06T21:50:00           2.0          0   
2     

In [232]:
# Normalización básica
for c in ['ts_ini','ts_fin']:
    if c in ops.columns:
        ops[c] = pd.to_datetime(ops[c], errors='coerce')
if 'duracion_min' in ops.columns:
    ops['duracion_min'] = pd.to_numeric(ops['duracion_min'], errors='coerce')

# Ref a string
for df in [header, ops]:
    if 'ref_id' in df.columns:
        df['ref_id_str'] = df['ref_id'].astype(str).str.replace(r'\.0$','', regex=True).str.zfill(6)
    if 'work_order_id' in df.columns:
        df['work_order_id'] = df['work_order_id'].astype(str)

# Unir info del header a operaciones
cols_header = ['work_order_id','ref_id_str']
for c in ['qty_plan','qty_estimado','fecha_lanzamiento','due_date']:
    if c in header.columns:
        cols_header.append(c)
ops = ops.merge(header[cols_header], on='work_order_id', how='left', suffixes=('','_hdr'))
print('Ops + header shape:', ops.shape)
print(ops[['work_order_id','ref_id_str','op_id','op_text','machine_id','ts_ini','duracion_min']].head())


Ops + header shape: (65170, 28)
  work_order_id ref_id_str         op_id       op_text  machine_id              ts_ini  duracion_min
0       24/0658     081000  SOLDADURA-RE  SOLDADURA-RE         515 2025-01-06 21:47:00           3.0
1       24/0658     081000  SOLDADURA-RE  SOLDADURA-RE         515 2025-01-06 21:48:00           2.0
2       24/0674     124203   RECTIFICADO   RECTIFICADO        1001 2025-01-06 21:49:00         500.0
3       24/0725     904802  SOLDADURA-RE  SOLDADURA-RE         503 2025-01-06 21:49:00         486.0
4       24/0731     081147  SOLDADURA-RE  SOLDADURA-RE         505 2025-01-06 21:49:00         485.0


In [233]:
# Catálogo de operaciones
print('Top op_id:')
print(ops['op_id'].value_counts().head(20))
print('Top op_text:')
print(ops['op_text'].value_counts().head(20))

Top op_id:
op_id
RECTIFICADO     12053
TALLADO         11400
SOLDADURA-RE    10411
CALIBRADO        8189
ENTRADA DE D     4919
TEMPLE-REVEN     4087
TORNEADO         2887
CURVADO-CORT     2666
NORMALIZADO      1302
GRANALLADO       1275
SACAR ENTRAD      704
TORNEAR INTE      503
TEMPLAR POR       491
TALLAR            414
APLANADO          323
SOLDAR-REBAB      301
CALIBRAR FRI      279
GRANALLADO E      220
FRESADO           165
CURVAR-CORTA       84
Name: count, dtype: int64
Top op_text:
op_text
RECTIFICADO     12053
TALLADO         11400
SOLDADURA-RE    10411
CALIBRADO        8189
ENTRADA DE D     4919
TEMPLE-REVEN     4087
TORNEADO         2887
CURVADO-CORT     2666
NORMALIZADO      1302
GRANALLADO       1275
SACAR ENTRAD      704
TORNEAR INTE      503
TEMPLAR POR       491
TALLAR            414
APLANADO          323
SOLDAR-REBAB      301
CALIBRAR FRI      279
GRANALLADO E      220
FRESADO           165
CURVAR-CORTA       84
Name: count, dtype: int64


In [234]:
# Fase: usar op_id (o op_text si falta) y caer a evento
ops['fase_inferida'] = (ops['op_id']
                        .fillna(ops['op_text'])
                        .fillna(ops['evento'])
                        .astype(str)
                        .str.strip())
ops['fase_inferida'] = ops['fase_inferida'].replace('', 'SIN_OP')
print(ops[['op_id','op_text','evento','fase_inferida']].head())


          op_id       op_text      evento fase_inferida
0  SOLDADURA-RE  SOLDADURA-RE  Producción  SOLDADURA-RE
1  SOLDADURA-RE  SOLDADURA-RE  Incidencia  SOLDADURA-RE
2   RECTIFICADO   RECTIFICADO  Producción   RECTIFICADO
3  SOLDADURA-RE  SOLDADURA-RE  Producción  SOLDADURA-RE
4  SOLDADURA-RE  SOLDADURA-RE  Producción  SOLDADURA-RE


In [235]:
# Secuencia por orden
ops_sorted = ops.sort_values(['work_order_id','ts_ini','ts_fin'])

registros = []
for wo, g in ops_sorted.groupby('work_order_id'):
    fases = g['fase_inferida'].tolist()
    ops_seq = g['op_id'].tolist()
    dur_total = g['duracion_min'].sum(skipna=True)
    ref = g['ref_id_str'].iloc[0]
    start = fases[0] if fases else None
    end = fases[-1] if fases else None
    registros.append({
        'work_order_id': wo,
        'ref_id_str': ref,
        'fase_inicio': start,
        'fase_fin': end,
        'ruta_fases': ' -> '.join(fases),
        'dur_total_min': dur_total,
        'n_ops': len(fases),
        'ops_seq': ops_seq,
    })

routes_df = pd.DataFrame(registros)
print('Órdenes con ruta reconstruida:', routes_df.shape)
routes_df.head()


Órdenes con ruta reconstruida: (622, 8)


,work_order_id,ref_id_str,fase_inicio,fase_fin,ruta_fases,dur_total_min,n_ops,ops_seq
0,20/4428,109800,NORMALIZADO,CURVADO-CORT,NORMALIZADO -> TEMPLE-REVEN -> GRANALLADO -> T...,1721.0,25,"[NORMALIZADO, TEMPLE-REVEN, GRANALLADO, TALLAD..."
1,22/1009,003501,TEMPLAR POR,TORNEAR Ø EX,TEMPLAR POR -> TEMPLAR POR -> TEMPLAR POR -> T...,2481.0,33,"[TEMPLAR POR, TEMPLAR POR, TEMPLAR POR, TEMPLA..."
2,23/0113,118001,TALLADO,TALLADO,TALLADO -> TALLADO -> TALLADO -> TALLADO -> In...,856.0,16,"[TALLADO, TALLADO, TALLADO, TALLADO, nan, TALL..."
3,23/0172,191000,CALIBRAR FRI,CALIBRAR FRI,CALIBRAR FRI,0.0,1,[CALIBRAR FRI]
4,23/0366,818003,SOLDADURA-RE,SOLDADURA-RE,SOLDADURA-RE -> SOLDADURA-RE -> SOLDADURA-RE -...,450.0,8,"[SOLDADURA-RE, SOLDADURA-RE, SOLDADURA-RE, SOL..."


In [236]:
# Inicio y fin de fase por orden
start_end_counts = routes_df.groupby(['fase_inicio','fase_fin']).size().sort_values(ascending=False)
print('Top combinaciones inicio->fin:')
print(start_end_counts.head(15))

# Estadísticas generales
routes_df['n_ops'] = routes_df['n_ops'].fillna(0)
print('Resumen duración total (min) y nº ops por orden:')
print(routes_df[['dur_total_min','n_ops']].describe())


Top combinaciones inicio->fin:
fase_inicio   fase_fin    
CURVADO-CORT  RECTIFICADO     167
SOLDADURA-RE  RECTIFICADO     102
NORMALIZADO   RECTIFICADO      36
CURVADO-CORT  TEMPLE-REVEN     31
              CALIBRADO        25
GRANALLADO    RECTIFICADO      22
TALLADO       RECTIFICADO      16
RECTIFICADO   RECTIFICADO      11
CURVADO-CORT  TALLADO          11
              TORNEADO          9
              CURVADO-CORT      9
CALIBRADO     RECTIFICADO       8
CURVADO-CORT  ENTRADA DE D      8
SOLDADURA-RE  SOLDADURA-RE      8
CURVADO-CORT  SOLDADURA-RE      8
dtype: int64
Resumen duración total (min) y nº ops por orden:
       dur_total_min      n_ops
count     622.000000  622.00000
mean    12421.272026  104.77492
std     10096.593283   81.12042
min         0.000000    1.00000
25%      5187.250000   48.00000
50%     10587.500000   89.00000
75%     17375.250000  141.75000
max     62788.000000  546.00000


#### Resumen por orden: fases únicas y piezas

Para cada OF: lista de fases (sin repetir), sumas de piezas_ok y scrap, y piezas lanzadas (ok + scrap).

In [237]:
# Resumen por OF
resumen_rows = []
for wo, g in ops_sorted.groupby('work_order_id'):
    fases_unicas = list(dict.fromkeys(g['fase_inferida'].tolist()))  # preserva orden, sin repetir
    piezas_ok = g['piezas_ok'].sum(skipna=True) if 'piezas_ok' in g else np.nan
    piezas_scrap = g['piezas_scrap'].sum(skipna=True) if 'piezas_scrap' in g else np.nan
    piezas_lanzadas = piezas_ok + piezas_scrap if pd.notna(piezas_ok) and pd.notna(piezas_scrap) else np.nan
    resumen_rows.append({
        'work_order_id': wo,
        'ref_id_str': g['ref_id_str'].iloc[0],
        'fases_unicas': ' -> '.join(fases_unicas),
        'n_fases': len(fases_unicas),
        'piezas_ok_sum': piezas_ok,
        'piezas_scrap_sum': piezas_scrap,
        'piezas_lanzadas': piezas_lanzadas
    })
resumen_df = pd.DataFrame(resumen_rows)
print('Resumen por OF (primeras 10):')
print(resumen_df.head(10))


Resumen por OF (primeras 10):
  work_order_id ref_id_str                                       fases_unicas  n_fases  piezas_ok_sum  piezas_scrap_sum  piezas_lanzadas
0       20/4428     109800  NORMALIZADO -> TEMPLE-REVEN -> GRANALLADO -> T...        8           9056                 0             9056
1       22/1009     003501  TEMPLAR POR -> SACAR ENTRAD -> Incidencia -> T...        4            868                31              899
2       23/0113     118001                 TALLADO -> Incidencia -> CALIBRADO        3           4062                 0             4062
3       23/0172     191000                                       CALIBRAR FRI        1              0                 0                0
4       23/0366     818003                                       SOLDADURA-RE        1           9466                14             9480
5       23/0378     010002                              TALLADO -> Incidencia        2            556                16              572
6       23/

#### Dataset de flujo por OF

Para cada orden: ruta (fases únicas en orden), piezas lanzadas/ok/scrap, duración total y el flujo por fase (piezas agregadas por fase sin repetir).

In [238]:
# Excluir OF que no estén en el header (sin qty_plan)
header_wos = set(header['work_order_id'].astype(str)) if 'header' in globals() else set()

flows = []

# Función para obtener flujo de una OF (solo si empieza en CURV*)
def get_flow_for_order(df_order):
    # Separar incidencias/averías/preparación
    mask_incid = df_order['evento'].str.contains('incid', case=False, na=False) if 'evento' in df_order else False
    mask_incid = mask_incid | (df_order['tipo_incidencia'].notna() if 'tipo_incidencia' in df_order else False)
    df_prod = df_order.loc[~mask_incid].copy()
    df_inc = df_order.loc[mask_incid].copy()

    if df_prod.empty:
        return None

    fases_seq = df_prod['fase_inferida'].tolist()
    if not fases_seq:
        return None
    first = fases_seq[0]
    if not str(first).upper().startswith('CURV'):
        return None

    # Cantidad lanzada: qty_plan (header) si existe, si no suma ok+scrap
    qty_plan_val = df_prod['qty_plan'].iloc[0] if 'qty_plan' in df_prod.columns else None
    if pd.isna(qty_plan_val):
        qty_plan_val = None
    base_lanzada = qty_plan_val
    if base_lanzada is None:
        base_lanzada = df_prod[['piezas_ok','piezas_scrap']].sum().sum()

    primera_fase_ts = df_prod['ts_ini'].min() if 'ts_ini' in df_prod.columns else None
    ultima_fase_ts = df_prod['ts_fin'].max() if 'ts_fin' in df_prod.columns else None
    fabricacion_duracion_min = None
    if pd.notna(primera_fase_ts) and pd.notna(ultima_fase_ts):
        fabricacion_duracion_min = (ultima_fase_ts - primera_fase_ts).total_seconds() / 60.0

    incidencias_duracion_total = df_inc['duracion_min'].sum(skipna=True) if not df_inc.empty else 0

    # Agregado por fase (suma scrap, duración). Piezas_ok no se usa para flujo porque suma duplicados.
    agg_phase = (df_prod.groupby('fase_inferida')
                   .agg(piezas_scrap_sum=('piezas_scrap','sum'),
                        duracion_min_sum=('duracion_min','sum'))
                   .reset_index())

    # Orden por aparición en la secuencia completa
    order_map = {f:i for i,f in enumerate(fases_seq)}
    agg_phase['orden'] = agg_phase['fase_inferida'].map(order_map)
    agg_phase = agg_phase.sort_values('orden')

    # Flujo: entrada inicial = base_lanzada; salida = entrada - scrap_fase
    rem = base_lanzada
    flow_list = []
    scrap_total_calc = 0
    for _, row in agg_phase.iterrows():
        entrada = rem
        scrap = row['piezas_scrap_sum'] if pd.notna(row['piezas_scrap_sum']) else 0
        salida = None
        if pd.notna(entrada):
            salida = max(entrada - scrap, 0)
            rem = salida
        scrap_total_calc += scrap
        flow_list.append({
            'fase': row['fase_inferida'],
            'entrada_estimada': entrada,
            'salida_estimada': salida,
            'piezas_scrap': scrap,
            'duracion_min': row['duracion_min_sum']
        })

    piezas_ok_total = rem
    dur_total = df_prod['duracion_min'].sum(skipna=True)

    return {
        'fases_seq': fases_seq,
        'fases_unicas': list(dict.fromkeys(fases_seq)),
        'flow_por_fase': flow_list,
        'piezas_ok_total': piezas_ok_total,
        'piezas_scrap_total': scrap_total_calc,
        'piezas_lanzadas': base_lanzada,
        'duracion_total_min': dur_total,
        'primera_fase_ts': primera_fase_ts,
        'ultima_fase_ts': ultima_fase_ts,
        'fabricacion_duracion_min': fabricacion_duracion_min,
        'incidencias_duracion_total': incidencias_duracion_total,
    }

for wo, g in ops_sorted.groupby('work_order_id'):
    if header_wos and wo not in header_wos:
        continue
    flow = get_flow_for_order(g)
    if flow is None:
        continue
    flows.append({
        'work_order_id': wo,
        'ref_id_str': g['ref_id_str'].iloc[0],
        'fases_seq': flow['fases_seq'],
        'fases_unicas': flow['fases_unicas'],
        'n_ops': len(flow['fases_seq']),
        'piezas_lanzadas': flow['piezas_lanzadas'],
        'piezas_ok_total': flow['piezas_ok_total'],
        'piezas_scrap_total': flow['piezas_scrap_total'],
        'duracion_total_min': flow['duracion_total_min'],
        'primera_fase_ts': flow['primera_fase_ts'],
        'ultima_fase_ts': flow['ultima_fase_ts'],
        'fabricacion_duracion_min': flow['fabricacion_duracion_min'],
        'incidencias_duracion_total': flow['incidencias_duracion_total'],
        'flow_por_fase': flow['flow_por_fase'],
    })

flow_df = pd.DataFrame(flows)
print('Resumen de flujo por OF (head):')
print(flow_df[['work_order_id','ref_id_str','n_ops','piezas_lanzadas','piezas_ok_total','piezas_scrap_total','duracion_total_min']].head())

# Crear columnas OP1..OPN según longitud máxima
def expand_ops(df):
    if df.empty:
        return df
    max_len = df['fases_seq'].str.len().max()
    cols = [f'OP{i+1}' for i in range(max_len)]
    data = []
    for seq in df['fases_seq']:
        row = seq + [None]*(max_len-len(seq))
        data.append(row)
    ops_expanded = pd.DataFrame(data, columns=cols)
    return df.reset_index(drop=True).join(ops_expanded)

flow_df_expanded = expand_ops(flow_df)
print('Tabla expandida con columnas OP1..OPN (head):')
print(flow_df_expanded[[c for c in flow_df_expanded.columns if c.startswith('OP')]].head())

# Mostrar detalle de la primera OF como ejemplo
if not flow_df.empty:
    ejemplo = flow_df.iloc[0]
    print(f"Detalle de flujo por fase - OF {ejemplo['work_order_id']} ref {ejemplo['ref_id_str']}")
    display(pd.DataFrame(ejemplo['flow_por_fase']))


Resumen de flujo por OF (head):
  work_order_id ref_id_str  n_ops  piezas_lanzadas  piezas_ok_total  piezas_scrap_total  duracion_total_min
0       24/0712     031100    102           9753.0           9620.0                 133             32153.0
1       24/0722     008091     54           2596.0           2525.0                  71             11508.0
2       24/0728     092901     36           1000.0            953.0                  47              3435.0
3       24/0733     124203    118           9247.0           9149.0                  98             41450.0
4       24/0734     124203     37           2639.0           2614.0                  25             11135.0
Tabla expandida con columnas OP1..OPN (head):
            OP1           OP2           OP3           OP4           OP5           OP6           OP7           OP8           OP9          OP10          OP11          OP12          OP13          OP14  \
0  CURVADO-CORT  CURVADO-CORT  CURVADO-CORT  SOLDADURA-RE  SOLDADURA-RE  

,fase,entrada_estimada,salida_estimada,piezas_scrap,duracion_min
0,CURVADO-CORT,9753.0,9723.0,30,1565.0
1,SOLDADURA-RE,9723.0,9707.0,16,3340.0
2,NORMALIZADO,9707.0,9707.0,0,1026.0
3,GRANALLADO,9707.0,9707.0,0,1028.0
4,CALIBRADO,9707.0,9707.0,0,1299.0
5,TALLADO,9707.0,9685.0,22,12167.0
6,ENTRADA DE D,9685.0,9664.0,21,6124.0
7,RECTIFICADO,9664.0,9620.0,44,5604.0


In [239]:
# Ejemplos de órdenes con inicio/fin y ruta
print(routes_df[['work_order_id','ref_id_str','fase_inicio','fase_fin','n_ops','dur_total_min','ruta_fases']].head(10))


  work_order_id ref_id_str   fase_inicio      fase_fin  n_ops  dur_total_min                                         ruta_fases
0       20/4428     109800   NORMALIZADO  CURVADO-CORT     25         1721.0  NORMALIZADO -> TEMPLE-REVEN -> GRANALLADO -> T...
1       22/1009     003501   TEMPLAR POR  TORNEAR Ø EX     33         2481.0  TEMPLAR POR -> TEMPLAR POR -> TEMPLAR POR -> T...
2       23/0113     118001       TALLADO       TALLADO     16          856.0  TALLADO -> TALLADO -> TALLADO -> TALLADO -> In...
3       23/0172     191000  CALIBRAR FRI  CALIBRAR FRI      1            0.0                                       CALIBRAR FRI
4       23/0366     818003  SOLDADURA-RE  SOLDADURA-RE      8          450.0  SOLDADURA-RE -> SOLDADURA-RE -> SOLDADURA-RE -...
5       23/0378     010002       TALLADO       TALLADO      5          708.0  TALLADO -> TALLADO -> TALLADO -> Incidencia ->...
6       23/0421     091503       TALLADO       TALLADO      1            0.0                            

#### Consulta puntual por OF

Introduce una `work_order_id` y obtén su ruta (fases únicas), piezas buenas/scrap/lanzadas y la secuencia completa de operaciones.

In [240]:
# Consulta por OF con agregados por fase
consulta_of = '24/0708'  # cambia a la OF que quieras

g = ops_sorted[ops_sorted['work_order_id'] == consulta_of]
if g.empty:
    print(f'No se encontró la OF {consulta_of}')
else:
    fases_unicas = list(dict.fromkeys(g['fase_inferida'].tolist()))
    piezas_ok = g['piezas_ok'].sum(skipna=True) if 'piezas_ok' in g else np.nan
    piezas_scrap = g['piezas_scrap'].sum(skipna=True) if 'piezas_scrap' in g else np.nan
    piezas_lanzadas = piezas_ok + piezas_scrap if pd.notna(piezas_ok) and pd.notna(piezas_scrap) else np.nan
    dur_total = g['duracion_min'].sum(skipna=True)
    print(f"OF {consulta_of} | ref {g['ref_id_str'].iloc[0]} | fases: {' -> '.join(fases_unicas)}")
    print(f"Piezas lanzadas: {piezas_lanzadas}, ok: {piezas_ok}, scrap: {piezas_scrap}, duración total (min): {dur_total}")

    # Agregado por fase (sin repetir filas)
    agg = (g.groupby('fase_inferida')
             .agg(piezas_ok_sum=('piezas_ok','sum'),
                  piezas_scrap_sum=('piezas_scrap','sum'),
                  duracion_min_sum=('duracion_min','sum'))
             .reset_index())
    agg['piezas_lanzadas'] = agg['piezas_ok_sum'] + agg['piezas_scrap_sum']
    display(agg)
    
    # Secuencia completa (por si se quiere detalle)
    display(g[['op_id','op_text','fase_inferida','ts_ini','ts_fin','duracion_min','piezas_ok','piezas_scrap']])


OF 24/0708 | ref 473205 | fases: CURVADO-CORT -> SOLDADURA-RE -> Incidencia -> GRANALLADO E -> CALIBRADO -> NORMALIZADO -> TORNEADO -> TALLADO -> TEMPLE-REVEN
Piezas lanzadas: 39366, ok: 39349, scrap: 17, duración total (min): 8773.0


,fase_inferida,piezas_ok_sum,piezas_scrap_sum,duracion_min_sum,piezas_lanzadas
0,CALIBRADO,13380,0,771.0,13380
1,CURVADO-CORT,5920,0,287.0,5920
2,GRANALLADO E,12920,0,317.0,12920
3,Incidencia,0,0,26.0,0
4,NORMALIZADO,2877,0,524.0,2877
5,SOLDADURA-RE,1274,12,1352.0,1286
6,TALLADO,1012,0,3031.0,1012
7,TEMPLE-REVEN,976,4,1058.0,980
8,TORNEADO,990,1,1407.0,991


,op_id,op_text,fase_inferida,ts_ini,ts_fin,duracion_min,piezas_ok,piezas_scrap
172,CURVADO-CORT,CURVADO-CORT,CURVADO-CORT,2025-01-07 13:51:00,2025-01-07 18:01:00,250.0,5920,0
216,CURVADO-CORT,CURVADO-CORT,CURVADO-CORT,2025-01-07 15:26:00,2025-01-07 15:47:00,21.0,0,0
240,SOLDADURA-RE,SOLDADURA-RE,SOLDADURA-RE,2025-01-07 16:58:00,2025-01-07 18:22:00,84.0,5,3
243,CURVADO-CORT,CURVADO-CORT,CURVADO-CORT,2025-01-07 17:06:00,2025-01-07 17:22:00,16.0,0,0
278,SOLDADURA-RE,SOLDADURA-RE,SOLDADURA-RE,2025-01-07 18:22:00,2025-01-07 21:57:00,215.0,264,0
...,...,...,...,...,...,...,...,...
6267,TORNEADO,TORNEADO,TORNEADO,2025-01-24 09:18:00,2025-01-24 09:37:00,19.0,0,0
6274,TALLADO,TALLADO,TALLADO,2025-01-24 09:38:00,2025-01-24 11:26:00,108.0,33,0
6306,TEMPLE-REVEN,TEMPLE-REVEN,TEMPLE-REVEN,2025-01-24 12:00:00,2025-01-24 13:57:00,117.0,164,2
6317,TEMPLE-REVEN,TEMPLE-REVEN,TEMPLE-REVEN,2025-01-24 12:57:00,2025-01-24 13:03:00,6.0,0,0


#### Flujo de vida en formato JSON por OF

Construye un objeto por OF con: orden, referencia, cantidad lanzada, lista de operaciones (OP1..OPn con piezas buenas/scrap), cantidad final, scrap total y tiempo total.

In [241]:
import json

def to_iso(val):
    if hasattr(val, 'isoformat'):
        return val.isoformat()
    return val

flows_json = []
for _, row in flow_df.iterrows():
    ops_list = []
    seen = set()
    for fase in row['fases_seq']:
        if fase in seen:
            continue
        seen.add(fase)
        info = next((f for f in row['flow_por_fase'] if f['fase'] == fase), None)
        if info is None:
            continue
        ops_list.append({
            'op': fase,
            'piezas_lanzadas': float(info['entrada_estimada']) if info['entrada_estimada'] is not None else None,
            'salida_estimada': float(info['salida_estimada']) if info['salida_estimada'] is not None else None,
            'piezas_scrap': float(info['piezas_scrap']) if info['piezas_scrap'] is not None else None,
            'duracion_min': float(info['duracion_min']) if info['duracion_min'] is not None else None,
        })
    flows_json.append({
        'work_order_id': row['work_order_id'],
        'ref_id_str': row['ref_id_str'],
        'cantidad_lanzada': float(row['piezas_lanzadas']) if row['piezas_lanzadas'] is not None else None,
        'operaciones': ops_list,
        'cantidad_final_ok': float(row['piezas_ok_total']) if row['piezas_ok_total'] is not None else None,
        'scrap_total': float(row['piezas_scrap_total']) if row['piezas_scrap_total'] is not None else None,
        'tiempo_total_min': float(row['duracion_total_min']) if row['duracion_total_min'] is not None else None,
        'primera_fase_ts': to_iso(row.get('primera_fase_ts')),
        'ultima_fase_ts': to_iso(row.get('ultima_fase_ts')),
        'fabricacion_duracion_min': float(row['fabricacion_duracion_min']) if row.get('fabricacion_duracion_min') is not None else None,
        'incidencias_duracion_total': float(row['incidencias_duracion_total']) if row.get('incidencias_duracion_total') is not None else None,
    })

print('Ejemplo de flujo JSON (primeros 2):')
print(json.dumps(flows_json[:2], indent=2, ensure_ascii=False))

out_dir = Path('../data/processed')
out_dir.mkdir(parents=True, exist_ok=True)
json_path = out_dir / 'of_flujos_curvado.json'
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(flows_json, f, ensure_ascii=False, indent=2)
print('Exportado a', json_path)


Ejemplo de flujo JSON (primeros 2):
[
  {
    "work_order_id": "24/0712",
    "ref_id_str": "031100",
    "cantidad_lanzada": 9753.0,
    "operaciones": [
      {
        "op": "CURVADO-CORT",
        "piezas_lanzadas": 9753.0,
        "salida_estimada": 9723.0,
        "piezas_scrap": 30.0,
        "duracion_min": 1565.0
      },
      {
        "op": "SOLDADURA-RE",
        "piezas_lanzadas": 9723.0,
        "salida_estimada": 9707.0,
        "piezas_scrap": 16.0,
        "duracion_min": 3340.0
      },
      {
        "op": "NORMALIZADO",
        "piezas_lanzadas": 9707.0,
        "salida_estimada": 9707.0,
        "piezas_scrap": 0.0,
        "duracion_min": 1026.0
      },
      {
        "op": "GRANALLADO",
        "piezas_lanzadas": 9707.0,
        "salida_estimada": 9707.0,
        "piezas_scrap": 0.0,
        "duracion_min": 1028.0
      },
      {
        "op": "CALIBRADO",
        "piezas_lanzadas": 9707.0,
        "salida_estimada": 9707.0,
        "piezas_scrap": 0.0,
    